In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import difflib
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies_data = pd.read_csv('/content/movies.csv')

In [3]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
movies_data.shape

(4803, 24)

In [5]:
selected_features = ['genres', 'keywords', 'overview', 'tagline', 'cast', 'director']  #extra overview and we will not train on title we will use it
print(selected_features)

['genres', 'keywords', 'overview', 'tagline', 'cast', 'director']


In [6]:
movies_data.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0


Here in title there are 0 null values and according to me we should drop rows(moveis) wilth null title

In [7]:
for feature in selected_features: #replace null values for those selected_features only
  movies_data[feature] = movies_data[feature].fillna('')

In [8]:
movies_data.isnull().sum()

,0
index,0
budget,0
genres,0
homepage,3091
id,0
keywords,0
original_language,0
original_title,0
overview,0
popularity,0


In [9]:
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['overview'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']

In [10]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      When ambitious New York attorney Sam is sent...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [11]:
combined_features.shape

(4803,)

In [12]:
vectorizer = TfidfVectorizer()

In [13]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [14]:
print(feature_vectors)

  (0, 561)	0.05971816344971169
  (0, 703)	0.06846420517510078
  (0, 9754)	0.08513696797398294
  (0, 23977)	0.07941905576010944
  (0, 10023)	0.07960231361105431
  (0, 6601)	0.1498786462809525
  (0, 5279)	0.1549075340655008
  (0, 10796)	0.11095111375730655
  (0, 25413)	0.24369151759694266
  (0, 29469)	0.08637114089261566
  (0, 5592)	0.17322386697661618
  (0, 25232)	0.1336739112380023
  (0, 13474)	0.03647840985795873
  (0, 27118)	0.08328687324810813
  (0, 239)	0.19716936546022962
  (0, 4768)	0.12501550204808315
  (0, 20104)	0.19260031442011954
  (0, 17021)	0.14324534514274753
  (0, 14023)	0.041700928584839175
  (0, 7827)	0.1797988202434295
  (0, 27405)	0.0310150192195571
  (0, 18249)	0.14769810641761003
  (0, 20039)	0.37747447361884223
  (0, 19541)	0.04908322232312576
  (0, 28597)	0.1549075340655008
  :	:
  (4802, 9588)	0.10415492266309316
  (4802, 21386)	0.08975511036766048
  (4802, 9166)	0.07829183421073847
  (4802, 24102)	0.08207980733784753
  (4802, 6136)	0.10774075928383217
  (4802, 

In [15]:
similarity = cosine_similarity(feature_vectors)

In [16]:
print(similarity)

[[1.         0.05083168 0.0332947  ... 0.02749812 0.0304889  0.0072518 ]
 [0.05083168 1.         0.04356836 ... 0.05077045 0.03100979 0.01521198]
 [0.0332947  0.04356836 1.         ... 0.02646984 0.04751623 0.01372603]
 ...
 [0.02749812 0.05077045 0.02646984 ... 1.         0.03481447 0.03546821]
 [0.0304889  0.03100979 0.04751623 ... 0.03481447 1.         0.03098945]
 [0.0072518  0.01521198 0.01372603 ... 0.03546821 0.03098945 1.        ]]


In [17]:
similarity.shape

(4803, 4803)

In [18]:
movie_name = input('Enter movie name : ')

Enter movie name : Iron man


In [19]:
list_of_all_titles = movies_data['title']
print(list_of_all_titles)

0                                         Avatar
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
                          ...                   
4798                                 El Mariachi
4799                                   Newlyweds
4800                   Signed, Sealed, Delivered
4801                            Shanghai Calling
4802                           My Date with Drew
Name: title, Length: 4803, dtype: object


In [20]:
close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

In [21]:
print(close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [22]:
closest_match = close_match[0]
print(closest_match)

Iron Man


In [23]:
index_of_the_movie = movies_data[movies_data.title == closest_match]['index'].values[0]
print(index_of_the_movie)

68


In [24]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.05017469339266746), (1, 0.03650601760111055), (2, 0.01102894045774897), (3, 0.006566052540268532), (4, 0.02626006458279672), (5, 0.014352343583526092), (6, 0.032839932646772305), (7, 0.1934346428669005), (8, 0.010447641591762173), (9, 0.03661745849142617), (10, 0.03856206332307925), (11, 0.013140335946520337), (12, 0.010480204139154865), (13, 0.010519252119405036), (14, 0.05824054285547451), (15, 0.02850607017788164), (16, 0.13208839109747222), (17, 0.01578360213400237), (18, 0.020783067282324263), (19, 0.04248644812075494), (20, 0.036752005155219786), (21, 0.024462379417016164), (22, 0.009211246725242385), (23, 0.017688499354673565), (24, 0.017920651785080483), (25, 0.00351667593485752), (26, 0.11563787911286949), (27, 0.03304503555209013), (28, 0.037268071590691204), (29, 0.009557912877353868), (30, 0.07969083021504449), (31, 0.2905211440511144), (32, 0.02575250044213903), (33, 0.07770036840942475), (34, 0.005794060102999554), (35, 0.020964205751412706), (36, 0.025304022864516

In [25]:
sorted_similar_movies = sorted(similarity_score, key = lambda x : x[1], reverse = True)
print(sorted_similar_movies)

[(68, 1.0000000000000002), (79, 0.32928784708836273), (31, 0.2905211440511144), (7, 0.1934346428669005), (16, 0.13208839109747222), (511, 0.11735356375367764), (4401, 0.11644899793001534), (26, 0.11563787911286949), (64, 0.11260372706734423), (182, 0.10817936918053872), (3623, 0.10427579910401667), (94, 0.10008582067970911), (46, 0.08808497545952931), (4033, 0.0865367140010388), (783, 0.08640309672854926), (174, 0.08596763747178215), (618, 0.08422372953874693), (85, 0.08317925570431418), (3133, 0.08003408207059728), (101, 0.07997032837045653), (1740, 0.07985500618953217), (30, 0.07969083021504449), (3466, 0.0786385267898631), (33, 0.07770036840942475), (203, 0.07717739012595405), (1177, 0.07676994814946042), (420, 0.07660248386797326), (131, 0.07526258364935168), (1406, 0.07520922477471761), (318, 0.07470160419701297), (788, 0.07388756055635984), (2229, 0.07375196657625921), (122, 0.07319454323849185), (1135, 0.07304073502421528), (2235, 0.07248187289157068), (138, 0.07246371735320548)

In [31]:
print('Movie sugggestion for you : \n')
i = 1
n = int(input('Enter number of sugestion : '))
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index == index]['title'].values[0]
  i += 1
  print(title_from_index)
  if(i > n):
    break

Movie sugggestion for you : 

Enter number of sugestion : 30
Iron Man
Iron Man 2
Iron Man 3
Avengers: Age of Ultron
The Avengers
X-Men
The Helix... Loaded
Captain America: Civil War
X-Men: Apocalypse
Ant-Man
Made
Guardians of the Galaxy
X-Men: Days of Future Past
Super
Mortdecai
The Incredible Hulk
Mystery Men
Captain America: The Winter Soldier
The Devil's Tomb
X-Men: First Class
Kick-Ass 2
Spider-Man 2
Sliding Doors
X-Men: The Last Stand
X2
Sin City
Hellboy II: The Golden Army
G-Force
The Nativity Story
Surf's Up
